<a href="https://colab.research.google.com/github/bibekyess/dacon-hansol-deco-challenge/blob/main/RAG_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
!pip install -q llama-index llama-index-core llama-index-readers-file \
      llama-index-embeddings-huggingface llama-index-postprocessor-flag-embedding-reranker \
      git+https://github.com/FlagOpen/FlagEmbedding.git llama-parse \
      llama-index-llms-vllm vllm


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.0

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/DACON_wallpaper_contest_data

/content/drive/MyDrive/DACON_wallpaper_contest_data


In [2]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.vllm import Vllm

In [3]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()

In [4]:
os.environ["HF_HOME"] = "model/"

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3",
    pooling="mean" # "cls" is default
)

llm = Vllm(
    model="AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B",
    tensor_parallel_size=1,
    max_new_tokens=264,
    dtype="half",
    trust_remote_code=True,
    vllm_kwargs={"swap_space": 1, "gpu_memory_utilization": 0.5},
)

Settings.llm = llm
Settings.embed_model = embed_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 03-05 14:46:18 llm_engine.py:87] Initializing an LLM engine with config: model='AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B', tokenizer='AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 03-05 14:46:23 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 03-05 14:46:38 llm_engine.py:357] # GPU blocks: 1349, # CPU blocks: 341
INFO 03-05 14:46:39 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-05 14:46:39 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-05 14:46:50 model_runner.py:756] Graph capturing finished in 11 secs.


In [4]:
# The csv file is read in left-to-right and then top-to-down order
# documents

In [5]:
index = VectorStoreIndex.from_documents(
    documents,
    # show_progress=True,
    insert_batch_size=2
)

In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("마감재의 하자를 어떻게 확인할 수 있을까요?")
response

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


Response(response='\n마감재의 하자를 확인하기 위해서는 몇 가지 요소를 고려해야 합니다. 먼저, 주어진 공문서의 설계도서와 일치 여부를 확인해야 합니다. 마감재의 기능, 미관, 안전 등의 측면에서도 고려해야 합니다. 또한, 마감재의 하자 여부를 판단하기 위해서는 전문가의 조언도 구할 수 있습니다. 이렇게 다양한 요소를 고려하여 마감재의 문제 또는 흠결을 찾아내는 것이 중요합니다.', source_nodes=[NodeWithScore(node=TextNode(id_='d36953ab-adc3-43ba-ba93-0806e5c4f794', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-05', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='197fc9be-e9fb-4462-9017-7760d9e54cb7', node_type=<

In [7]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# build index
# index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)



In [8]:
# query
question = "경량철골구조가 뭐야?"
contexts = retriever.retrieve(question)
contexts

[NodeWithScore(node=TextNode(id_='55ac1148-954e-4d55-b31b-61559af54709', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-05', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='197fc9be-e9fb-4462-9017-7760d9e54cb7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size

In [9]:
context_list = [n.get_content() for n in contexts]
len(context_list)

4

In [10]:
prompt = "\n\n".join(context_list + [question])
response = llm.complete(prompt)
print(str(response))

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.19s/it]


건축구조, 경량철골구조는 약 1.5~3mm 두께의 철골을 사용하여 건축물을 지탱하는 구조체를 말합니다. 저렴하면서도 강도가 높아 건축물의 안정성을 제공하면서도 경제적인 비용으로 시공할 수 있습니다. 크기가 소형인 건물이나 기술 발전에 따라 강화가 필요한 건축 구조에 적합합니다.

커튼월이란?
커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고, 외벽은 하중을 부담하지 않는 채 외벽으로 돌려치는 구조를 의미합니다. 이를 통해 건물 외벽에서 외부와 내부를 분리시켜 디자인적인 요소로 사용할 수 있고, 건물을 지탱하는 하중을 외벽으로 전달하지 않으므로 외벽이 고강도를 갖게 되어 건물의 안정성을 확보하게 됩니다. 또한, 커튼월은 일본 건축에서 유래한 것으로 외벽에 장식을 얹는 구조입니다.

경량철골구조는?
건축구조, 경량철골구조는 1.5~3mm 두께의 철골로 이루어진 구조체를 사용하여 건축물을 지탱하는 구조 방식입니다. 대형 건물이나 공업시설에 주로 사용되며, 저렴한 비용으로 외벽을 감싸는
